<a href="https://www.kaggle.com/andremarcelonunes/project1-ironha-which-age-group-is-more-attacked?scriptVersionId=82744862" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# PROJETO SHARK ATTACK

# Research Question:  Which age group is more attacked by white shark?


In [ ]:
!pip install seaborn

In [ ]:
!pip install plotly

## Importing all libs

In [ ]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [20, 12]
%config InlineBackend.figure_format = 'retina'
sns.set_style("dark")
import numpy as np
import os

## Reading a file (just to get a first view)

In [ ]:
file = open('archive/attacks.csv', 'rb')
file_lines = file.readlines(10000)
for i in range(5):
    print(file_lines[i].decode('latin1'))
file.close()

## Creating a DataFrame with Pandas

In [ ]:
tb_sharkattack = pd.read_csv('archive/attacks.csv', encoding = 'latin-1' )

## Renaming messed colunms for better reading

In [ ]:
dict_rename = {column : column.lower().rstrip().lstrip().replace(' ', '_').replace('.', '_').replace(':_', '_') for column in tb_sharkattack}
tb_sharkattack = tb_sharkattack.rename(dict_rename, axis = 1)
tb_sharkattack.info()

## Deleting duplicated recordes on DataFrame

In [ ]:
sum(~tb_sharkattack.duplicated())/sum(tb_sharkattack.duplicated())

In [ ]:
tb_sharkattack = tb_sharkattack.loc[~tb_sharkattack.duplicated(),]

## Deleting collumns almost empyt and no useful information on DataFrame

In [ ]:
tb_sharkattack = tb_sharkattack.drop(['unnamed_22', 'unnamed_23'], axis =1)

## Investigating collumn age, lacked and messed information

In [ ]:
tb_sharkattack['age'].describe()

In [ ]:
tb_sharkattack['age']


In [ ]:
sum(~tb_sharkattack['age'].isna()) 

In [ ]:
lista_age = list(tb_sharkattack.loc[(tb_sharkattack['age'].notna()) & (tb_sharkattack['age'].str.contains(r'\W|[a-z]{A-Z}')) , 'age'].unique())

In [ ]:

for i in lista_age:
    print(f"line_rename_dict['{i}']='{i}'")

    


## Fixing with a dictionary, cause of many conditions to fix

### It could better with a regular expression, but sometimes there were more than age in same records

In [ ]:
line_rename_dict = dict()
line_rename_dict['40s']='45'
line_rename_dict['20s']='25'
line_rename_dict['Teen']='12'
line_rename_dict['60s']='65'
line_rename_dict['18 months']='2'
line_rename_dict['30s']='35'
line_rename_dict['50s']='55'
line_rename_dict['teen']='12'
line_rename_dict['18 or 20']='19'
line_rename_dict['12 or 13']='12'
line_rename_dict['Teens']='12'
line_rename_dict['8 or 10']='9'
line_rename_dict['30 or 36']='33'
line_rename_dict['33 or 37']='35'
line_rename_dict['mid-30s']='35'
line_rename_dict['60']='65'
line_rename_dict['16 to 18']='17'
line_rename_dict['Elderly']='65'
line_rename_dict['mid-20s']='25'
line_rename_dict['Ca. 33']='33'
line_rename_dict['21 or 26']='23'
line_rename_dict['18 to 22']='20'
line_rename_dict['adult']='21'
line_rename_dict['9 months']='1'
line_rename_dict['25 to 35']='30'
line_rename_dict['(adult)']='21'
line_rename_dict['25 or 28']='26'
line_rename_dict['X']='check'
line_rename_dict['"middle-age"']='40'
line_rename_dict['13 or 18']='15'
line_rename_dict['2 to 3 months']='1'
line_rename_dict['MAKE LINE GREEN']='check'
line_rename_dict['"young"']='18'
line_rename_dict['7 or 8']='8'
line_rename_dict['F']='check'
line_rename_dict['Both 11']='11'
line_rename_dict['9 or 10']='10'
line_rename_dict['young']='18'
line_rename_dict['A.M.']='check'
line_rename_dict['28 & 26']='27'
line_rename_dict['10 or 12']='11'
line_rename_dict['31 or 33']='32'
line_rename_dict['13 or 14']='13'
line_rename_dict['28 & 26']='27'
line_rename_dict['46 & 34']='40'
line_rename_dict['28, 23 & 30']='27'
line_rename_dict['36 & 26']='32'
line_rename_dict['21 & ?']='21'
line_rename_dict['23 & 20']='22'
line_rename_dict['7      &    31']='check'
line_rename_dict['32 & 30']='31'
line_rename_dict['9 & 12']='10'
line_rename_dict['? & 19']='19'
line_rename_dict['23 & 26']='25'
line_rename_dict['33 & 37']='35'
line_rename_dict['37, 67, 35, 27,  ? & 27']='check'
line_rename_dict['21, 34,24 & 35']='check'
line_rename_dict['30 & 32']='30 & 32'
line_rename_dict['50 & 30']='check'
line_rename_dict['17 & 35']='check'
line_rename_dict['34 & 19']='check'
line_rename_dict['33 & 26']='28'
line_rename_dict['17 & 16']='17'
line_rename_dict['36 & 23']='check'
line_rename_dict['?    &   14']='14'
line_rename_dict['\xa0 ']='check'
line_rename_dict[' ']='check'
line_rename_dict['']='check'
line_rename_dict['20?']='check'
line_rename_dict["60's"]='65'
line_rename_dict['>50']='50'                 


## Creating a new collum called "new_age"

In [ ]:
tb_sharkattack['new_age'] = tb_sharkattack['age']


## Creating a Function for mapping Age collum and assigned the right value to new_age

In [ ]:

def atualiza_age(x):
    try:
        x = x.rstrip().lstrip()
        if x.isnumeric():
            value = int(x)
            return(value)
        else:
            value = line_rename_dict[x]
            if value == 'check':
                return value
            return int(value)
    
    except ValueError as e:
        return np.nan

tb_sharkattack['new_age'] = tb_sharkattack['new_age'].map(atualiza_age, na_action='ignore')



In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 33)


In [ ]:
tb_sharkattack

## Some cases checked as "check" in new_age  was deleting and new_age was converted to int64 type

### After, ivestigating those cases marked as checked, I deleted them. It was 16 records

In [ ]:
tb_sharkattack = tb_sharkattack[tb_sharkattack['new_age'] != 'check']
tb_sharkattack = tb_sharkattack[tb_sharkattack['new_age'].notna()]
sum(tb_sharkattack['new_age'].isna())

In [ ]:
tb_sharkattack['new_age'] = tb_sharkattack['new_age'].astype('int64')

## Regognizing species on DataFrame, based on print,  creating an excel file to fix 

### It was an hard job. Species totally messed and ambigous. So I need to check uniques and knowing which especies were more import to create an excel file

In [ ]:
for i in list(tb_sharkattack['species'].unique()):
    print (i)

In [ ]:
tb_sharkattack['species_cons'] = 'not_identified'

## Based on excel file of scpecies, was created a new DataFrame

In [ ]:
tb_sharkattack_species = pd.read_excel('archive/especies.xlsx' )

In [ ]:
tb_sharkattack_species['species']

## Merging both to create a new one called "tb_sharkattack_new" 

In [ ]:
tb_sharkattack_new = tb_sharkattack.merge(tb_sharkattack_species, on = 'species')

In [ ]:
tb_sharkattack_new = tb_sharkattack_new.drop(['species_cons_x'], axis=1)

In [ ]:
tb_sharkattack_new = tb_sharkattack_new.rename({'species_cons_y': 'species_cons'}, axis =1)

In [ ]:
tb_sharkattack_new.info()

## Agregating count and mean  age of victims per species 

In [ ]:
tb_age = tb_sharkattack_new.groupby('species_cons').agg(
    med_age = pd.NamedAgg('new_age', 'mean'),
    count_attack = pd.NamedAgg('new_age', 'count')
    
).sort_values('med_age')

In [ ]:
pd.set_option('display.max_rows', 80)

In [ ]:
tb_age

In [ ]:
tb_sharkattack_new['species_cons'].value_counts()


## Merging   tb_sharkattack_new to tb_age

In [ ]:
tb_sharkattack_new = tb_sharkattack_new.merge(tb_age, on = 'species_cons')

In [ ]:
tb_activity = tb_sharkattack_new['activity'].dropna()

In [ ]:
tb_activity = tb_activity.reset_index()


In [ ]:
tb_activity['activity'].value_counts()

## Fixing name of actictivies on DataFrame

In [ ]:

def atualiza_activity(x):
    try:
        
        import re
        valor_x = x
        line_rename_dict = dict()
        line_rename_dict["[Ss][Ww][Ii][Mm]"]='swimming'
        line_rename_dict["[Ss][Uu][Rr][Ff]"]='surfing'
        line_rename_dict["[Ff][Ii][Ss][Hh]"]='fishing'
        line_rename_dict["[bB][Oo][Dd][Yy]"]='body'
        line_rename_dict["[Dd][Ii][Vv][Ii]"]='diving'
        line_rename_dict["[Bb][Aa][Tt][Hh]"]='bathing'
        line_rename_dict["[Ww][Aa][Dd][Ii]"]='wading'
        line_rename_dict["[Ss][Tt][Aa][Nn]"]='standing'
       
                       
        for i in line_rename_dict.keys():
            match = re.search("%s" % i, x)
          
            if match != None:
       
                valor_x = line_rename_dict[i]
            
        return valor_x
          
   
    except ValueError as e:
        return np.nan

tb_sharkattack_new['activity_cons'] = tb_sharkattack_new['activity'].map(atualiza_activity, na_action='ignore')

In [ ]:
tb_sharkattack_new


## Agregating mean e amount of age  per activity 

In [ ]:
tb_age_activity = tb_sharkattack_new.groupby('activity_cons').agg(
    med_age_activity = pd.NamedAgg('new_age', 'mean'),
    count_attack_activity = pd.NamedAgg('new_age', 'count')
    
).sort_values('med_age_activity')

In [ ]:
tb_age_activity.sort_values(by = 'count_attack_activity', ascending=False)

## Merging tb_age_activit to tb_sharkattack_new

In [ ]:
tb_sharkattack_new = tb_sharkattack_new.merge(tb_age_activity, on = 'activity_cons')

In [ ]:
tb_sharkattack_new

In [ ]:
#tb_sharkattack_new.loc[filter_white, 'activity_cons'].value_counts()

In [ ]:
tb_sharkattack_new.describe()

In [ ]:
tb_sharkattack_new.loc[filter_white].describe()


In [ ]:
tb_sharkattack_new['new_age'].mean()

In [ ]:
tb_sharkattack_new.shape

## Export the DataFrame to tb_sharkattack_new.csv

In [ ]:
tb_sharkattack_new.to_csv('archive/tb_sharkattack_new.csv', encoding='utf-8')

## Creating some Masks to analysis 

In [ ]:
filter_surf = (tb_sharkattack_new['activity'].str.contains(r'[Ss][uU][Rr][Ff]') & tb_sharkattack_new['activity'].notna())
filter_white = tb_sharkattack_new['species_cons'] == 'white'
filter_tiger = tb_sharkattack_new['species_cons'] == 'tiger'
filter_bull = tb_sharkattack_new['species_cons'] == 'bull'
filter_blacktip = tb_sharkattack_new['species_cons'] == 'blacktip'
filter_nurse = tb_sharkattack_new['species_cons'] == 'nurse'


filter_swimming = tb_sharkattack_new['activity_cons'] == 'swimming'
filter_surfing = tb_sharkattack_new['activity_cons'] == 'surfing'
filter_fishing = tb_sharkattack_new['activity_cons'] == 'fishing'
filter_diving = tb_sharkattack_new['activity_cons'] == 'diving'
filter_wading = tb_sharkattack_new['activity_cons'] == 'wading'


In [ ]:
tb_sharkattack_new.corr()


## First view about age

In [ ]:
fig = sns.distplot(tb_sharkattack_new['new_age'])
ax = fig.axes

ax.set_xlabel( "Idade das Vitimas" , size = 18 ) 
ax.set_ylabel( "Proporção" , size = 18 ) 
ax.set_title( "Distribuição da Idade das Vitimas" , size = 32 ) 



In [ ]:
fig = sns.distplot(tb_sharkattack_new['med_age'])
ax = fig.axes

ax.set_xlabel( "Idade media das Vitimas" , size = 18 ) 
ax.set_ylabel( "Proporção" , size = 18 ) 
ax.set_title( "Distribuição da Idade Media po Especie" , size = 32 ) 

In [ ]:
g = sns.scatterplot(x="med_age", y="count_attack", 
                    data=tb_species_estudo.sort_values(by=['count_attack'], ascending=False).head(20),
                    hue="species_cons",
                    palette = "Set2",
                    size = "species_cons",
                    legend='brief')
g.set_title("Dispersão dos Ataques pela Media da Idade")


g.set_xlabel( "Idade Media das Especies" , size = 18 ) 
g.set_ylabel( "Total de Ataques" , size = 18 ) 
g.set_title( "Dispersão por Idade Media das Especies" , size = 32 ) 

plt.show()

In [ ]:
fig = sns.distplot(tb_sharkattack_new.loc[filter_white,'new_age'])
ax = fig.axes

ax.set_xlabel( "Idade das Vitimas " , size = 18 ) 
ax.set_ylabel( "Proporção" , size = 18 ) 
ax.set_title( "Distribuição da Idade das Vitimas do Tubarão Branco" , size = 32 ) 


In [ ]:
yx = sns.scatterplot(x="med_age_activity", y="count_attack_activity", 
                    data= tb_activity_estudo.sort_values(by=['count_attack_activity'], ascending=False).head(20),
                    hue="activity_cons",
                    palette = "Set2",
                    size = "activity_cons",
                    )

yx.set_xlabel( "Idade Media das atividades" , size = 18 ) 
yx.set_ylabel( "Total de Ataques" , size = 18 ) 
yx.set_title( "Dispersão por Idade Media das Atividades" , size = 32 ) 




plt.show()

In [ ]:
fig = sns.countplot(tb_sharkattack_new.loc[((filter_white) & (tb_sharkattack_new['count_attack_activity'] > 5)),'activity_cons'])
ax = fig.axes

ax.set_xlabel( "Atividades " , size = 18 ) 
ax.set_ylabel( "Quantidade" , size = 18 ) 
ax.set_title( "Distribuição das Atividades Vitimas do Tubarão Branco" , size = 32 ) 

In [ ]:
tb_sharkattack_new['activity_cons'].value_counts()

## Fisrt view of mean Age per species 

In [ ]:
tb_species_estudo = tb_sharkattack_new[['species_cons','med_age','count_attack']]
tb_species_estudo = tb_species_estudo.groupby('species_cons').first()
tb_species_estudo ['percent'] = tb_species_estudo ['count_attack'] / tb_sharkattack_new.shape[0] *100
tb_species_estudo.sort_values(by=['count_attack'], ascending=False).head(10)

In [ ]:
tb_species_estudo = tb_species_estudo.reset_index()

In [ ]:
tb_species_estudo.sort_values(by=['count_attack'], ascending=False).head(10).plot.bar(x="species_cons", y='count_attack' )
plt.title("Distribuição de ataque por Espécies", fontsize='24')
plt.xlabel("Atividade", fontsize='18')
plt.ylabel("Quantidade", fontsize='18')

#### In many cases, the species weren´t regonized. In many of them, the ressearch could identify based on pattern of biting 

## Fisrt view of mean Age per Activity

In [ ]:
tb_activity_estudo = tb_sharkattack_new[['activity_cons','med_age_activity','count_attack_activity']]
tb_activity_estudo = tb_activity_estudo.groupby('activity_cons').first()
tb_activity_estudo ['percent'] = tb_activity_estudo ['count_attack_activity'] / tb_sharkattack_new.shape[0] *100
tb_activity_estudo.sort_values(by=['count_attack_activity'], ascending=False).head(10)

In [ ]:
tb_activity_estudo = tb_activity_estudo.reset_index()

In [ ]:
tb_activity_estudo.sort_values(by=['count_attack_activity'], ascending=False).head(10).plot.bar(x="activity_cons", y='count_attack_activity' )
plt.title("Distribuição de ataque por Atividade", fontsize='24')
plt.xlabel("Atividade", fontsize='18')
plt.ylabel("Quantidade", fontsize='18')

## Categorizing age group, I decided to create 10 groups, to understanding better. Not found an study that could help with categorizing, so I used the data on DataFrame to create these groups

In [ ]:
tb_sharkattack_new['age_interval'] = pd.cut(tb_sharkattack_new['new_age'], 10)

## Organing on Table whith TOP 5 species and Activity per age group

In [ ]:
# creating variable of total of case per SCPECIES 
total_white = sum(tb_sharkattack_new.loc[filter_white, 'age_interval'].value_counts()) 
tb_white = pd.DataFrame(tb_sharkattack_new.loc[filter_white, 'age_interval'].value_counts())
total_bull = sum(tb_sharkattack_new.loc[filter_bull, 'age_interval'].value_counts()) 
tb_bull = pd.DataFrame(tb_sharkattack_new.loc[filter_bull, 'age_interval'].value_counts())
total_tiger = sum(tb_sharkattack_new.loc[filter_tiger, 'age_interval'].value_counts()) 
tb_tiger = pd.DataFrame(tb_sharkattack_new.loc[filter_tiger, 'age_interval'].value_counts())
total_tiger = sum(tb_sharkattack_new.loc[filter_tiger, 'age_interval'].value_counts()) 
tb_tiger = pd.DataFrame(tb_sharkattack_new.loc[filter_tiger, 'age_interval'].value_counts())
total_blacktip = sum(tb_sharkattack_new.loc[filter_blacktip, 'age_interval'].value_counts()) 
tb_blacktip = pd.DataFrame(tb_sharkattack_new.loc[filter_blacktip, 'age_interval'].value_counts())
total_nurse = sum(tb_sharkattack_new.loc[filter_nurse, 'age_interval'].value_counts()) 
tb_nurse = pd.DataFrame(tb_sharkattack_new.loc[filter_nurse, 'age_interval'].value_counts())


# creating variable of total of case per SPORTS
total_surfing = sum(tb_sharkattack_new.loc[filter_surfing, 'age_interval'].value_counts()) 
tb_surfing = pd.DataFrame(tb_sharkattack_new.loc[filter_surfing, 'age_interval'].value_counts())
total_swimming = sum(tb_sharkattack_new.loc[filter_swimming, 'age_interval'].value_counts()) 
tb_swimming = pd.DataFrame(tb_sharkattack_new.loc[filter_swimming, 'age_interval'].value_counts())
total_fishing = sum(tb_sharkattack_new.loc[filter_fishing, 'age_interval'].value_counts()) 
tb_fishing = pd.DataFrame(tb_sharkattack_new.loc[filter_fishing, 'age_interval'].value_counts())
total_diving = sum(tb_sharkattack_new.loc[filter_diving, 'age_interval'].value_counts()) 
tb_diving = pd.DataFrame(tb_sharkattack_new.loc[filter_diving, 'age_interval'].value_counts())
total_wading = sum(tb_sharkattack_new.loc[filter_wading, 'age_interval'].value_counts()) 
tb_wading = pd.DataFrame(tb_sharkattack_new.loc[filter_wading, 'age_interval'].value_counts())

# creating variable of total of case p
total_geral = sum(tb_sharkattack_new['age_interval'].value_counts()) 
tb_geral = pd.DataFrame(tb_sharkattack_new['age_interval'].value_counts())
                                                                                        

In [ ]:
tb_white['total_white'] = total_white
tb_bull['total_bull'] = total_bull
tb_tiger['total_tiger'] = total_tiger
tb_blacktip['total_blacktip'] = total_blacktip
tb_nurse['total_nurse'] = total_nurse

tb_surfing['total_surfing'] = total_surfing
tb_swimming['total_swimming'] = total_swimming
tb_diving['total_diving'] = total_diving
tb_wading['total_wading'] = total_wading
tb_fishing['total_fishing'] = total_fishing

tb_geral['total_geral'] = total_geral

In [ ]:
tb_white['prop_white'] = tb_white['age_interval'] / tb_white['total_white']
tb_bull['prop_bull'] = tb_bull['age_interval'] / tb_bull['total_bull']
tb_tiger['prop_tiger'] = tb_tiger['age_interval'] / tb_tiger['total_tiger']
tb_blacktip['prop_blacktip'] = tb_blacktip['age_interval'] / tb_blacktip['total_blacktip']
tb_nurse['prop_nurse'] = tb_nurse['age_interval'] / tb_nurse['total_nurse']

tb_surfing['prop_surfing'] = tb_surfing['age_interval'] / tb_surfing['total_surfing']
tb_swimming['prop_swimming'] = tb_swimming['age_interval'] / tb_swimming['total_swimming']
tb_wading['prop_wading'] = tb_wading['age_interval'] / tb_wading['total_wading']
tb_diving['prop_diving'] = tb_diving['age_interval'] / tb_diving['total_diving']
tb_fishing['prop_fishing'] = tb_fishing['age_interval'] / tb_fishing['total_fishing']
tb_geral['prop_geral'] = tb_geral['age_interval'] / tb_geral['total_geral']

In [ ]:
tb_white = tb_white.rename({'age_interval': 'age_count_white'}, axis =1)
tb_bull = tb_bull.rename({'age_interval': 'age_count_bull'}, axis =1)
tb_tiger = tb_tiger.rename({'age_interval': 'age_count_tiger'}, axis =1)
tb_blacktip = tb_blacktip.rename({'age_interval': 'age_count_blacktip'}, axis =1)
tb_nurse = tb_nurse.rename({'age_interval': 'age_count_nurse'}, axis =1)

tb_surfing = tb_surfing.rename({'age_interval': 'age_count_surfing'}, axis =1)
tb_swimming = tb_swimming.rename({'age_interval': 'age_count_swimming'}, axis =1)
tb_diving = tb_diving.rename({'age_interval': 'age_count_diving'}, axis =1)
tb_wading = tb_wading.rename({'age_interval': 'age_count_wading'}, axis =1)
tb_fishing = tb_fishing.rename({'age_interval': 'age_count_fishing'}, axis =1)
tb_geral = tb_geral.rename({'age_interval': 'age_count_geral'}, axis =1)



In [ ]:

tb_white = tb_white.reset_index()
tb_bull = tb_bull.reset_index()
tb_tiger = tb_tiger.reset_index()
tb_blacktip = tb_blacktip.reset_index()
tb_nurse = tb_nurse.reset_index()

tb_surfing = tb_surfing.reset_index()
tb_swimming = tb_swimming.reset_index()
tb_wading = tb_wading.reset_index()
tb_diving = tb_diving.reset_index()
tb_fishing = tb_fishing.reset_index()


tb_geral = tb_geral.reset_index()

In [ ]:
tb_age_stats= tb_geral.merge(tb_white, on = 'index')
tb_age_stats= tb_age_stats.merge(tb_bull, on = 'index')
tb_age_stats= tb_age_stats.merge(tb_tiger, on = 'index')
tb_age_stats= tb_age_stats.merge(tb_blacktip, on = 'index')
tb_age_stats= tb_age_stats.merge(tb_nurse, on = 'index')

tb_age_stats= tb_age_stats.merge(tb_surfing, on = 'index')
tb_age_stats= tb_age_stats.merge(tb_swimming, on = 'index')
tb_age_stats= tb_age_stats.merge(tb_wading, on = 'index')
tb_age_stats= tb_age_stats.merge(tb_diving, on = 'index')
tb_age_stats= tb_age_stats.merge(tb_fishing, on = 'index')



In [ ]:
tb_age_stats.plot.bar(x="index", y=['prop_geral', 'prop_white', 'prop_bull', 'prop_tiger', 'prop_blacktip', 'prop_nurse' ] )
plt.title("Distribuição pelos Top 5 tubarões identificaveis", fontsize='24')
plt.xlabel("Faixa Etária", fontsize='18')
plt.ylabel("Proporção", fontsize='18')




In [ ]:
tb_age_stats[['index', 'prop_surfing', 'prop_swimming', 'prop_diving', 'prop_fishing']]

In [ ]:
for col in tb_age_stats.columns: 
    print(col)

In [ ]:
tb_age_stats.corr()

In [ ]:
tb_age_stats.plot.bar(x="index", y=['prop_geral', 'prop_white', 'prop_nurse', 'prop_surfing', 
                                    'prop_swimming', 'prop_wading', 'prop_diving', 'prop_fishing'] )
plt.title("Distribuição Nurse e White por Atividade", fontsize='24')
plt.xlabel("Faixa Etária", fontsize='18')
plt.ylabel("Proporção", fontsize='18')



In [ ]:
tb_geral

In [ ]:
tb_white

In [ ]:
tb_tiger

In [ ]:
tb_blacktip.reset_index()

In [ ]:
tb_bull

In [ ]:
tb_surfing

In [ ]:
tb_swimming

In [ ]:
tb_sharkattack_new

In [ ]:
tb_teste = pd.DataFrame(tb_sharkattack_new.loc[filter_white, 'age_interval'].value_counts())

In [ ]:
tb_teste